In [2]:
# read the 2 files
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sp


df_drought = pd.read_csv('/home/mees/Desktop/forestfires_usa/dataset/drought_county_tx.csv')
df_fires = pd.read_csv('/home/mees/Desktop/forestfires_usa/dataset/texas_fires_with_fips.csv')

In [ ]:
# combine the drought and fire data using the FIPS code as the key. Use the value with the closest date
from tqdm import tqdm
import datetime

df_fires['DISCOVERY_DATE_NEW'] = pd.to_datetime(df_fires['DISCOVERY_DATE_NEW'])
df_drought['MapDate'] = pd.to_datetime(df_drought['MapDate'], format='%Y%m%d')

# to track the biggest date difference between all the combinations
biggest_date_diff = datetime.timedelta(0)

# calculate the min and max date difference between the fire and drought data
first_date = max(df_fires['DISCOVERY_DATE_NEW'].min(), df_drought['MapDate'].min())
last_date = min(df_fires['DISCOVERY_DATE_NEW'].max(), df_drought['MapDate'].max())
print('First date: ', first_date)
print('Last date: ', last_date)


df_fires['DSCI'] = np.nan
for i in tqdm(range(len(df_fires))):
    fips = df_fires.loc[i, 'FIPS']
    date = df_fires.loc[i, 'DISCOVERY_DATE_NEW']

    if date < first_date or date > last_date:
        continue

    drought_data = df_drought[df_drought['FIPS'] == fips]
    date_diff = abs(drought_data['MapDate'] - date)
    min_date_diff = date_diff.min()
    if min_date_diff > datetime.timedelta(weeks=10):
        print('Error: date difference is more than 10 weeks for FIPS code: ', fips)
    if min_date_diff > biggest_date_diff:
        biggest_date_diff = min_date_diff
        print('Biggest date difference so far: ', biggest_date_diff)

    drought_data = drought_data[date_diff == min_date_diff]
    if len(drought_data) == 1:
        dsci = drought_data['DSCI'].values[0]
        df_fires.loc[i, 'DSCI'] = dsci
    else:
        print('Error: multiple drought data found for FIPS code: ', fips)

df_fires.to_csv('/home/mees/Desktop/forestfires_usa/dataset/texas_fires_drought.csv', index=False)


First date:  2000-01-04 00:00:00
Last date:  2015-12-31 00:00:00


  0%|          | 48/142021 [00:00<04:57, 476.70it/s]

Biggest date difference so far:  1 days 00:00:00
Biggest date difference so far:  3 days 00:00:00


100%|██████████| 142021/142021 [03:58<00:00, 595.26it/s] 


In [7]:
# filter out all the rows that have no DSCI value
df_fires = df_fires.dropna(subset=['DSCI'])
# count rows
print('Number of rows with DSCI value: ', len(df_fires))
df_fires.to_csv('/home/mees/Desktop/forestfires_usa/dataset/texas_fires_drought.csv', index=False)


Number of rows with DSCI value:  129315
